## Vi arbejder med AKTIER &#129304;

### Importér nødvendige biblioteker

In [ ]:
import pandas as pd
import datetime
import numpy as np
from pandas_datareader import data as wb
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import yfinance as yf
%matplotlib inline

### Definér og download data: historiske aktiekurser for META, AMAZON og GOOGLE
- Prøv at "loade" andre aktier; gå ind på Yahoo Finance og find forkortelserne, fx Novo Nordisk og Mærsk?

In [ ]:
start = "2019-01-01"
end = '2022-1-01'
meta = yf.download('META',start,end)
googl = yf.download('GOOGL',start,end)
amzn = yf.download('AMZN',start,end)

### Undersøg dataene
- Hvad betyder tallet ud for "Length: " nedenfor?

In [ ]:
meta['Adj Close']

In [ ]:
print('Antal dage med børshandel i perioden 1/1-2019 til 1/1-2022:', len(meta['Adj Close']))

### Visualisér data
- Nedenfor hentes og bruges aktiekurserne statisk, dvs. én linje for hver; det kan gøres smartere med en løkke, men hvordan? :-)

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(meta['Adj Close'])
plt.plot(amzn['Adj Close'])
plt.ylabel('Returns')
plt.xlabel('Time')
plt.show()

### Data preprocessing: er der tomme pladser i datasættet? Overvej:
- Hvordan det påvirker resultatet?
- Hvad man kunne gøre med de tomme pladser?
- Brug nettet til at undersøge, hvordan man finder såkaldte "null"-pladser i en DataFrame i Python
    - Tjek, om der er "null"-pladser i datasættene

In [ ]:
# Kode her

### Data "preprocessing"
- Hvad gør nedenstående kode? Brug svarene fra spørgsmålene ovenfor :-)

In [ ]:
meta_data = {"meta": meta['Adj Close']}
meta_df = pd.DataFrame(meta_data)
meta_df['meta_normalized'] = (meta_df['meta'] - meta_df['meta'].min()) / (meta_df['meta'].max() - meta_df['meta'].min())

In [ ]:
amzn_data = {"amzn": amzn['Adj Close']}
amzn_df = pd.DataFrame(amzn_data)
amzn_df['amzn_normalized'] = (amzn_df['amzn'] - amzn_df['amzn'].min()) / (amzn_df['amzn'].max() - amzn_df['amzn'].min())

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(meta_df['meta_normalized'])
plt.plot(amzn_df['amzn_normalized'])
plt.ylabel('Returns')
plt.xlabel('Time')
plt.show()

### En konstruktiv måde at analysere udviklingen i aktiekuserne på er et såkaldt Moving Average (MA)
- Hvad dækker betegnelsen Moving Average over? Brug nettet til at finde ud af det ..

In [ ]:
meta['MA50'] = meta['Open'].rolling(50).mean()
meta['MA200'] = meta['Open'].rolling(200).mean()
meta['Open'].plot(figsize = (15,7))
meta['MA50'].plot()
meta['MA200'].plot()

### Undersøg en given akties volatilitet

In [ ]:
meta['returns'] = (meta['Close']/meta['Close'].shift(1)) -1
amzn['returns'] = (amzn['Close']/amzn['Close'].shift(1))-1
googl['returns'] = (googl['Close']/googl['Close'].shift(1)) - 1
meta['returns'].hist(bins = 100, label = 'Meta', alpha = 0.5, figsize = (15,7))
amzn['returns'].hist(bins = 100, label = 'Amazon', alpha = 0.5)
googl['returns'].hist(bins = 100, label = 'Google', alpha = 0.5)
plt.legend()

## Brug AI til at bygge en reel ML-model, der kan forudsige aktiekurser
Aktiekurser er som bekendt vældig volatile, og i modsætning til data med en stærk linæer sammenhæng, så er der meget, der kan påvirke udviklingen i kursen. Her et udpluk af nogle af de forholdsregler, man kan tage:

"Historical Influence: It's a common practice in time series forecasting (not just for stocks) to use a "window" of prior data points to predict the next one. The belief is that recent history might have some influence on the future value. In the realm of stocks, this could be due to momentum or other trading behaviors.
Practicality: A 60-day window is roughly equivalent to three months of trading days (considering that there are about 20-22 trading days in a month). This might capture quarterly financial results, which are significant events for publicly traded companies and can impact stock prices."

### Importér nødvendige biblioteker

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

### Indlæs data
#### Bemærk nedenfor:
- Der er brugt sekvenser à 60 dage... (som så også fungerer som <i>"features"</i>), men hvad er features i relation til ML-modeller?
- Brug din sidemakker til at forstå, hvad følgende linjer kode gør

In [ ]:
df = meta
df = df['Close'].values
df = df.reshape(-1, 1)

# Normalisér data
scaler = MinMaxScaler()
df = scaler.fit_transform(df)

x = []
y = []
for i in range(60, len(df)):
    x.append(df[i-60:i])
    y.append(df[i])

x = np.array(x)
y = np.array(y)

### Split dataene i træning/test

In [ ]:
split = int(0.8 * len(x))
x_train = x[:split]
y_train = y[:split]
x_test = x[split:]
y_test = y[split:]

### LSTM-model:
- Brug nettet til at finde ud af, hvad LSTM er for noget? Se eventuelt en YouTube-video ;-)
- Hvad er hyperparametre? 
    1) batch størrelse, 
    2) antal neuroner, 
    3) learning rate
- Prøv at lege med parametrene 0.3 og 100 nedenfor - hvad gør det for modellens præcision?

In [ ]:
model = Sequential()

model.add(LSTM(units=100, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.3))
model.add(LSTM(units=100, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(units=100))
model.add(Dropout(0.3))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')

#### Træn model på dataene:

In [ ]:
model.fit(x_train, y_train, epochs=50, batch_size=32)

#### Test og visualiser forudsigelser:

In [ ]:
y_pred = model.predict(x_test)
y_pred = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test)

plt.figure(figsize=(14,5))
plt.plot(y_test_inv, color='red', label='Rigtige aktiekurs')
plt.plot(y_pred, color='blue', label='Forudsagt aktiekurs')
plt.title('Forudsigelser af aktiekurser')
plt.xlabel('Tid')
plt.ylabel('Aktiekurs')
plt.legend()
plt.show()